## Загружаем word2vec модель

In [17]:
!unzip 187.zip

Archive:  187.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [18]:
!cat README

This archive is part of the NLPL Word Vectors Repository (http://vectors.nlpl.eu/repository/), version 1.1, published on Monday, October 15,2018.
Please see the file 'meta.json' in this archive and the overall repository metadata file http://vectors.nlpl.eu/repository/11.json for additional information.
The life-time identifier for this model is:
http://vectors.nlpl.eu/repository/11/187.zip

In [9]:
from gensim.models.keyedvectors import FastTextKeyedVectors

In [10]:
m = FastTextKeyedVectors.load('model.model')

## Что можно делать с эмбеддингами

Посмотрим размер словаря:

In [11]:
len(m.vocab)

192415

Часть эмбеддинга для 

In [17]:
m['исправляй'][:20]

array([-1.3566604 , -1.3532224 ,  0.9593475 , -1.8901982 , -0.59628433,
       -0.68555176,  2.905148  ,  1.9639157 , -0.32325912, -0.32197556,
        0.94435304,  2.501834  ,  0.40444824, -0.4573581 , -1.4188358 ,
       -1.8840065 , -1.5600121 , -0.5360782 , -0.1541971 , -0.702145  ],
      dtype=float32)

In [18]:
len(m['исправляй'])

300

Найти близкие:

In [28]:
m.most_similar('котик')

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('кошечка', 0.8114327788352966),
 ('котишка', 0.8018244504928589),
 ('котятка', 0.7937917113304138),
 ('котёнка', 0.784565806388855),
 ('котеночка', 0.7840605974197388),
 ('котка', 0.7780112624168396),
 ('котэ', 0.7776311635971069),
 ('котенка', 0.7739246487617493),
 ('енотик', 0.7660681009292603),
 ('котенок', 0.7625793218612671)]

Посчитать близость:

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
sim_matrix = cosine_similarity([m['кот']], [m['собака']])
sim_matrix

array([[0.6739326]], dtype=float32)

In [35]:
sim_matrix[0][0]

0.6739326

In [37]:
cosine_similarity([m['кот']], [m['котик']])[0][0]

0.73086

In [38]:
cosine_similarity([m['кот']], [m['крот']])[0][0]

0.5116138

In [39]:
cosine_similarity([m['Москва']], [m['город']])[0][0]

0.5540288

Сложить вектора:

In [49]:
something = m['Москва'] + m['Омск'] + m['Калуга']

In [50]:
cosine_similarity([something], [m['город']])

array([[0.63554007]], dtype=float32)

**Задание**: напишите функцию, которая принимает текст, а возвращает эмбеддинг этого текста, основанный на словах, входящих в состав.

In [51]:
from nltk.tokenize import word_tokenize

In [53]:
def embed_text(text):
    # your code
    

## Парафраз

Загрузим [данные](http://paraphraser.ru/download/get?file_id=1).

In [2]:
!unzip paraphraser.zip

Archive:  paraphraser.zip
  inflating: LICENSE                 
  inflating: corpus.xml              
  inflating: paraphrases.xml         


In [5]:
with open('paraphrases.xml') as f:
    data = f.read()

Как это выглядит:

In [7]:
!tail paraphrases.xml

      <value name="id">25577</value>
      <value name="id_1">34584</value>
      <value name="id_2">34722</value>
      <value name="text_1">9 мая метрополитен Петербурга будет работать круглосуточно</value>
      <value name="text_2">Мартынов: комендантский час в Донецке 9 мая будет отменен</value>
      <value name="jaccard">0.46188021535170065</value>
      <value name="class">-1</value>
    </paraphrase>
  </corpus>
</data>


### Создаём датасет для обучения

In [22]:
import pandas as pd

In [3]:
from xml.etree import ElementTree

In [19]:
root = ElementTree.XML(data)

In [20]:
par_data = {'text1': [], 'text2': [], 'class': []}
for par in root[1]:
    par_data['text1'].append(par[3].text)
    par_data['text2'].append(par[4].text)
    par_data['class'].append(int(par[6].text))

In [23]:
parphrase_df = pd.DataFrame(par_data)
parphrase_df.head(10)

,class,text1,text2
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...
5,1,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...
6,-1,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...
7,0,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...
8,-1,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...
9,-1,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...


А теперь давайте:
* векторизуем каждый документ
* посчитаем расстояние между каждой парой
* на этом обучим классификатор